Keras是一个基于Python编写的高层神经网络API，凭借用户友好性、模块化以及易扩展等有点大受好评，考虑到Keras的优良特性以及它的受欢迎程度，TensorFlow2.0中将Keras的代码吸收了进来，化身为tf.keras模块供用户使用。

使用tf.keras提供的高层API，可以轻松得完成建模三部曲——模型构建、训练、评估等工作。下面我们分别来说说如何使用tf.keras完成这三部曲。

# 1 模型构建

我们知道，神经网络模型就是层的堆叠，tf.keras提供的Sequential类对象就是层容器，可以轻松实现对层的堆叠，创建网络模型。用Sequential创建一个全连接网络模型：

In [1]:
import tensorflow as tf
from tensorflow import keras  # 为方便使用，keras一般单独导入
from tensorflow.keras import layers

In [2]:
model = tf.keras.Sequential()
# 往模型中添加一个有64个神经元组成的层，激活函数为relu:
model.add(layers.Dense(64, activation='relu'))
# 再添加一个:
model.add(layers.Dense(64, activation='relu'))
# 添加一个有10个神经元的softmax层作为输出层:
model.add(layers.Dense(10, activation='softmax'))

也可以在使用Sequential实例化模型时，通过传入由层组成的列表来添加层。我们换一种方式实现上面的模型构建过程,两种方式是完全等效的：

In [4]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')]
)

你看，用tf.keras创建一个模型，就是这么简单，只需要往Sequential中传入一个个tf.keras.layers定义的层就好了。进一步的，我们研究一下tf.keras.layers怎么个性化地创建层。

定义神经网络层通过tf.keras.layers模块中的Dense类实现，Dense类构造参数如下：
- units：指定神经元个数，必须是一个正整数。
- activation：激活函数，可以是可以是一个可调用对象或标识一个对象的字符串
- use_bias：布尔型，是否使用是否使用偏置项
- kernel_initializer和bias_initializer：权值、偏置初始化方法,可以是一个可调用对象或标识一个对象的字符串
- kernel_regularizer和bias_regularizer：对权值、偏置进行正则化的方法，可以是一个可调用对象或标识一个对象的字符串
- activity_regularizer：对层的输出进行正则化的方法，可以是一个可调用对象或标识一个对象的字符串
- kernel_constraint和bias_constraint：对权值矩阵、偏置矩阵的约束方法，可以是一个可调用对象或标识一个对象的字符串

In [6]:
# 有64个神经元，激活函数为sigmoid的层
layers.Dense(64, activation='sigmoid')
# 或者:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 对权值矩阵进行正则化:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 对偏置向量进行正则化:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 指定权值随机正交初始化:
layers.Dense(64, kernel_initializer='orthogonal')

# 指定偏置为常数:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

# 2 训练模型

## 2.1 配置：compile()

建立好模型之后，接下来当然是要进行训练模型了。不过，在训练前还需要做一些配置工作，例如指定优化器、损失函数、评估指标等，这些配置参数的过程一般通过tf.keras.Model.compile方法进行，先来熟悉一下tf.keras.Model.compile方法的三个常用参数：
- optimizer：tf.keras.optimizers模块中的优化器**实例化对象**，例如 tf.keras.optimizers.Adam或 tf.keras.optimizers.SGD的实例化对象，当然也可以使用字符串来指代优化器，例如'adam'和'sgd'。
- loss：损失函数，例如交叉熵、均方差等，通常是tf.keras.losses模块中定义的可调用对象，也可以用用于指代损失函数的字符串。
- metrics：元素为评估方法的list，通常是定义在tf.keras.metrics模块中定义的可调用对象，也可以用于指代评估方法的字符串。

在知道怎么配置模型训练参数后，就可以根据实际应用情况合理选择优化器、损失函数、评估方法等：

In [12]:
# 回归模型
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),  # 指定优化器，学习率为0.01
              loss='mse',       # 指定均方差作为损失函数
              metrics=['mae'])  # 添加绝对值误差作为评估方法

# 分类模型
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),  # 分类模型多用交叉熵作为损失函数
              metrics=[tf.keras.metrics.CategoricalAccuracy()])